In [1]:
# 0) Paths
from pathlib import Path
import pandas as pd
import numpy as np
from IPython.display import Markdown, display

PROJECT_ROOT = Path("..").resolve()
RES_DIR = PROJECT_ROOT / "results"
RES_DIR.mkdir(exist_ok=True, parents=True)

# File Paths
pd_path   = RES_DIR / "stress_pd_summary.csv"      # scenario, family, mean_pd, p50, p90, p99, uplift_vs_baseline_pct
el_path   = RES_DIR / "stress_el_summary.csv"      # scenario, family, mean_pd, EL, EL_change_vs_baseline_pct
macro_all_path = RES_DIR / "macro_scenarios_all.csv"  # data-driven + Fed scenarios (macro)
genai_path     = RES_DIR / "genai_stress_scenarios.csv"      # GenAI macro scenarios

In [2]:
# 1) Load risk tables

pd_df = pd.read_csv(pd_path)
el_df = pd.read_csv(el_path)
macro_all_df = pd.read_csv(macro_all_path)
genai_df = pd.read_csv(genai_path)

# Clean colnames (strip whitespace)
pd_df.columns = [c.strip() for c in pd_df.columns]
el_df.columns = [c.strip() for c in el_df.columns]

print("PD DF columns:", pd_df.columns.tolist())
print("EL DF columns:", el_df.columns.tolist())
print("Macro all DF columns:", macro_all_df.columns.tolist())
print("GenAI DF columns:", genai_df.columns.tolist())


PD DF columns: ['scenario', 'family', 'mean_pd', 'p50_pd', 'p90_pd', 'p99_pd', 'uplift_vs_baseline_pct']
EL DF columns: ['scenario', 'family', 'mean_pd', 'EL', 'EL_change_vs_baseline_pct']
Macro all DF columns: ['scenario', 'family', 'GDPC1', 'UNRATE', 'CPIAUCSL', 'FEDFUNDS', 'UNRATE_delta_qoq', 'FEDFUNDS_delta_qoq', 'GDPC1_delta_qoq', 'inflation_qoq', 'real_rate_qoq']
GenAI DF columns: ['scenario', 'GDPC1', 'UNRATE', 'CPIAUCSL', 'FEDFUNDS', 'GDPC1_delta_qoq', 'UNRATE_delta_qoq', 'CPIAUCSL_delta_qoq', 'FEDFUNDS_delta_qoq', 'inflation_qoq', 'real_rate_qoq']


In [3]:
el_df = el_df.drop(columns=["family", "mean_pd"])

In [4]:
# Ensure required columns exist
required_pd = {"scenario", "family", "mean_pd"}
required_el = {"scenario", "EL", "EL_change_vs_baseline_pct"}

missing_pd = required_pd - set(pd_df.columns)
missing_el = required_el - set(el_df.columns)
if missing_pd:
    raise ValueError(f"PD summary is missing columns: {missing_pd}")
if missing_el:
    raise ValueError(f"EL summary is missing columns: {missing_el}")

# Merge PD + EL on (scenario, family, mean_pd)
risk_df = pd.merge(
    pd_df,
    el_df[list(required_el)],  # keep only key + EL info
    on= "scenario",
    how="left",
    validate="one_to_one"
)

print("Merged risk_df shape:", risk_df.shape)
display(risk_df.head())

Merged risk_df shape: (16, 9)


,scenario,family,mean_pd,p50_pd,p90_pd,p99_pd,uplift_vs_baseline_pct,EL,EL_change_vs_baseline_pct
0,baseline_actual,baseline,0.205225,0.175645,0.391815,0.618134,0.000000,7.308635e+08,0.000000
1,mild_adverse,data_driven,0.264330,0.254242,0.418748,0.567447,28.799846,9.413511e+08,28.799846
2,severe_adverse,data_driven,0.152889,0.140718,0.257091,0.386563,-25.501673,5.444811e+08,-25.501673
3,Fed_Adverse,Fed,0.023423,0.019999,0.041342,0.072811,-88.586685,8.341576e+07,-88.586685
4,Fed_Baseline,Fed,0.242961,0.231942,0.389556,0.537474,18.387462,8.652508e+08,18.387462


In [5]:
# 2) Load macro scenario tables (Fed + data-driven + GenAI)

macro_df_list = []

# --- Load FED + DATA-DRIVEN macro scenarios ---
if macro_all_path.exists():
    macro_all = pd.read_csv(macro_all_path)
    macro_all.columns = macro_all.columns.str.strip()

    # Keep only macro features + scenario
    fed_macro_cols = [
        "scenario", "GDPC1", "UNRATE", "CPIAUCSL", "FEDFUNDS",
        "UNRATE_delta_qoq", "FEDFUNDS_delta_qoq",
        "GDPC1_delta_qoq", "inflation_qoq", "real_rate_qoq"
    ]
    macro_all = macro_all[fed_macro_cols]

    macro_all["macro_source"] = "macro_all"
    macro_df_list.append(macro_all)

# --- Load GenAI macro scenarios ---
if genai_path.exists():
    genai_macro = pd.read_csv(genai_path)
    genai_macro.columns = genai_macro.columns.str.strip()

    # Some GenAI tables include CPIAUCSL_delta_qoq; keep if exists
    genai_keep = [c for c in [
        "scenario", "GDPC1", "UNRATE", "CPIAUCSL", "FEDFUNDS",
        "UNRATE_delta_qoq", "FEDFUNDS_delta_qoq",
        "GDPC1_delta_qoq", "inflation_qoq", "real_rate_qoq",
        "CPIAUCSL_delta_qoq"  # may or may not exist
    ] if c in genai_macro.columns]

    genai_macro = genai_macro[genai_keep]
    genai_macro["macro_source"] = "GenAI"
    macro_df_list.append(genai_macro)

# === Clean + merge ===
if macro_df_list:
    # Combine all macro tables
    macro_df = pd.concat(macro_df_list, ignore_index=True)

    if "scenario" not in macro_df.columns:
        raise ValueError("Macro scenario tables must contain a 'scenario' column.")

    # If scenario appears both in macro_all and genAI → keep macro_all version
    macro_df = (
        macro_df.sort_values("macro_source")
                .drop_duplicates(subset=["scenario"], keep="first")
                .reset_index(drop=True)
    )

    # Identify macro feature columns
    macro_cols = [
        c for c in macro_df.columns
        if c not in {"scenario", "macro_source"}
    ]

    print("Macro columns:", macro_cols)

    # Final merge risk + macro
    full_df = pd.merge(
        risk_df,                          # PD + EL
        macro_df[["scenario"] + macro_cols], 
        on="scenario",
        how="left",
        validate="one_to_one"
    )

else:
    macro_cols = []
    full_df = risk_df.copy()
    print("No macro scenario files found; JSON will have empty 'macro' blocks.")

print("Final merged full_df shape:", full_df.shape)
display(full_df.head())


Macro columns: ['GDPC1', 'UNRATE', 'CPIAUCSL', 'FEDFUNDS', 'UNRATE_delta_qoq', 'FEDFUNDS_delta_qoq', 'GDPC1_delta_qoq', 'inflation_qoq', 'real_rate_qoq', 'CPIAUCSL_delta_qoq']
Final merged full_df shape: (16, 19)


,scenario,family,mean_pd,p50_pd,p90_pd,p99_pd,uplift_vs_baseline_pct,EL,EL_change_vs_baseline_pct,GDPC1,UNRATE,CPIAUCSL,FEDFUNDS,UNRATE_delta_qoq,FEDFUNDS_delta_qoq,GDPC1_delta_qoq,inflation_qoq,real_rate_qoq,CPIAUCSL_delta_qoq
0,baseline_actual,baseline,0.205225,0.175645,0.391815,0.618134,0.000000,7.308635e+08,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mild_adverse,data_driven,0.264330,0.254242,0.418748,0.567447,28.799846,9.413511e+08,28.799846,18782.243000,5.433333,247.238333,1.203333,-0.013072,0.202216,0.004002,0.006348,0.154176,NaN
2,severe_adverse,data_driven,0.152889,0.140718,0.257091,0.386563,-25.501673,5.444811e+08,-25.501673,17953.974000,6.933333,251.686333,1.923333,0.006803,0.555556,0.001845,0.007955,0.190550,NaN
3,Fed_Adverse,Fed,0.023423,0.019999,0.041342,0.072811,-88.586685,8.341576e+07,-88.586685,18821.441681,5.475000,257.030458,0.225000,0.800000,-0.500000,-0.035000,0.013000,-0.513000,NaN
4,Fed_Baseline,Fed,0.242961,0.231942,0.389556,0.537474,18.387462,8.652508e+08,18.387462,21214.070968,3.925000,260.107147,1.775000,0.000000,0.300000,0.028000,0.019000,0.281000,NaN


In [6]:

# 3) Build LLM-ready JSON

scenario_records = []

for _, row in full_df.iterrows():
    scen_name = str(row["scenario"])
    family    = str(row["family"])

    # ---- risk block ----
    risk_block = {
        "mean_pd": float(row["mean_pd"]),
    }
    # Optional PD distribution metrics if present
    for c in ["p50_pd", "p90_pd", "p99_pd", "uplift_vs_baseline_pct"]:
        if c in full_df.columns and pd.notna(row.get(c, None)):
            risk_block[c] = float(row[c])

    # EL info from EL summary
    if "EL" in full_df.columns and pd.notna(row.get("EL", None)):
        risk_block["EL"] = float(row["EL"])
    if "EL_change_vs_baseline_pct" in full_df.columns and pd.notna(row.get("EL_change_vs_baseline_pct", None)):
        risk_block["EL_change_vs_baseline_pct"] = float(row["EL_change_vs_baseline_pct"])

    # ---- macro block ----
    macro_block = {}
    for c in macro_cols:
        val = row.get(c, None)
        if pd.notna(val):
            # convert numpy -> Python scalar
            try:
                macro_block[c] = float(val)
            except Exception:
                macro_block[c] = val

    record = {
        "scenario": scen_name,
        "family": family,
        "macro": macro_block,
        "risk": risk_block,
    }
    scenario_records.append(record)


In [7]:
scenario_records

[{'scenario': 'baseline_actual',
  'family': 'baseline',
  'macro': {},
  'risk': {'mean_pd': 0.2052253071656771,
   'p50_pd': 0.1756445409627292,
   'p90_pd': 0.3918153723465106,
   'p99_pd': 0.6181335312891613,
   'uplift_vs_baseline_pct': 0.0,
   'EL': 730863518.4846855,
   'EL_change_vs_baseline_pct': 0.0}},
 {'scenario': 'mild_adverse',
  'family': 'data_driven',
  'macro': {'GDPC1': 18782.243,
   'UNRATE': 5.433333333333334,
   'CPIAUCSL': 247.23833333333337,
   'FEDFUNDS': 1.2033333333333334,
   'UNRATE_delta_qoq': -0.0130718954248367,
   'FEDFUNDS_delta_qoq': 0.2022160664819943,
   'GDPC1_delta_qoq': 0.0040024506125282,
   'inflation_qoq': 0.0063478253641482,
   'real_rate_qoq': 0.1541756763574446},
  'risk': {'mean_pd': 0.2643298804297704,
   'p50_pd': 0.2542419477959076,
   'p90_pd': 0.4187481942797376,
   'p99_pd': 0.5674474014341526,
   'uplift_vs_baseline_pct': 28.799846412888307,
   'EL': 941351089.2961066,
   'EL_change_vs_baseline_pct': 28.799846412888307}},
 {'scenario

In [8]:

# 4) Save JSON + flat CSV
import json
json_path = RES_DIR / "llm_scenarios.json"
flat_csv_path = RES_DIR / "llm_scenarios_flat.csv"

with json_path.open("w", encoding="utf-8") as f:
    json.dump(scenario_records, f, ensure_ascii=False, indent=2)

# Flatten for CSV inspection
flat_rows = []
for rec in scenario_records:
    base = {
        "scenario": rec["scenario"],
        "family": rec["family"],
        "mean_pd": rec["risk"].get("mean_pd"),
        "EL": rec["risk"].get("EL"),
        "EL_change_vs_baseline_pct": rec["risk"].get("EL_change_vs_baseline_pct"),
        "p50_pd": rec["risk"].get("p50_pd"),
        "p90_pd": rec["risk"].get("p90_pd"),
        "p99_pd": rec["risk"].get("p99_pd"),
        "uplift_vs_baseline_pct": rec["risk"].get("uplift_vs_baseline_pct"),
    }
    # Add macro_* columns
    for k, v in rec["macro"].items():
        base[f"macro_{k}"] = v
    flat_rows.append(base)

flat_df = pd.DataFrame(flat_rows)
flat_df.to_csv(flat_csv_path, index=False)

print(f"   Prepared {len(scenario_records)} scenarios for LLM.")


   Prepared 16 scenarios for LLM.


In [9]:
flat_df

,scenario,family,mean_pd,EL,EL_change_vs_baseline_pct,p50_pd,p90_pd,p99_pd,uplift_vs_baseline_pct,macro_GDPC1,macro_UNRATE,macro_CPIAUCSL,macro_FEDFUNDS,macro_UNRATE_delta_qoq,macro_FEDFUNDS_delta_qoq,macro_GDPC1_delta_qoq,macro_inflation_qoq,macro_real_rate_qoq,macro_CPIAUCSL_delta_qoq
0,baseline_actual,baseline,0.205225,7.308635e+08,0.000000,0.175645,0.391815,0.618134,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mild_adverse,data_driven,0.264330,9.413511e+08,28.799846,0.254242,0.418748,0.567447,28.799846,18782.243000,5.433333,247.238333,1.203333,-0.013072,0.202216,0.004002,0.006348,0.154176,NaN
2,severe_adverse,data_driven,0.152889,5.444811e+08,-25.501673,0.140718,0.257091,0.386563,-25.501673,17953.974000,6.933333,251.686333,1.923333,0.006803,0.555556,0.001845,0.007955,0.190550,NaN
3,Fed_Adverse,Fed,0.023423,8.341576e+07,-88.586685,0.019999,0.041342,0.072811,-88.586685,18821.441681,5.475000,257.030458,0.225000,0.800000,-0.500000,-0.035000,0.013000,-0.513000,NaN
4,Fed_Baseline,Fed,0.242961,8.652508e+08,18.387462,0.231942,0.389556,0.537474,18.387462,21214.070968,3.925000,260.107147,1.775000,0.000000,0.300000,0.028000,0.019000,0.281000,NaN
5,Fed_Severe,Fed,0.246729,8.786704e+08,20.223598,0.235858,0.394766,0.542903,20.223598,16907.187783,6.900000,254.741601,0.100000,1.500000,0.000000,-0.089000,0.009000,-0.009000,NaN
6,GenAI_S1,GenAI,0.404132,1.439226e+09,96.921346,0.404330,0.589221,0.723143,96.921346,18137.042969,6.846798,231.812775,0.407801,-0.012584,0.110715,0.009600,0.003911,0.106804,0.003911
7,GenAI_S2,GenAI,0.237731,8.466259e+08,15.839130,0.226518,0.382281,0.529835,15.839130,18122.355469,6.763257,232.605804,0.427591,-0.001771,-0.057966,0.003558,0.004170,-0.062136,0.004170
8,GenAI_S3,GenAI,0.339663,1.209632e+09,65.507294,0.334388,0.514944,0.659070,65.507294,18121.808594,6.804290,232.323074,0.426987,-0.011353,0.052106,0.005651,0.005674,0.046431,0.005674
9,GenAI_S4,GenAI,0.268575,9.564704e+08,30.868532,0.258698,0.424447,0.573174,30.868532,18112.789062,6.735481,232.800705,0.438021,-0.006777,-0.019390,0.001708,0.005047,-0.024437,0.005047


### LLM Result 

In [10]:
# Save flat_df :
# ['scenario', 'family', 'mean_pd', 'EL', 'EL_change_vs_baseline_pct',
#  'p50_pd', 'p90_pd', 'p99_pd', 'uplift_vs_baseline_pct',
#  'macro_GDPC1', 'macro_UNRATE', 'macro_CPIAUCSL', 'macro_FEDFUNDS', ... ]

RES_DIR = Path("../results")
llm_path = RES_DIR / "llm_narratives.json"

# Work on a copy so we don't mutate flat_df accidentally
merged_df = flat_df.copy()

print("Initial merged_df columns:")
print(merged_df.columns.tolist())


Initial merged_df columns:
['scenario', 'family', 'mean_pd', 'EL', 'EL_change_vs_baseline_pct', 'p50_pd', 'p90_pd', 'p99_pd', 'uplift_vs_baseline_pct', 'macro_GDPC1', 'macro_UNRATE', 'macro_CPIAUCSL', 'macro_FEDFUNDS', 'macro_UNRATE_delta_qoq', 'macro_FEDFUNDS_delta_qoq', 'macro_GDPC1_delta_qoq', 'macro_inflation_qoq', 'macro_real_rate_qoq', 'macro_CPIAUCSL_delta_qoq']


In [11]:
# === 1) Load LLM narratives JSON and merge =======================

if llm_path.exists():
    with open(llm_path, "r") as f:
        llm_data = json.load(f)

    llm_df = pd.DataFrame(llm_data)

    # Align columns with our scenario table
    # JSON example:
    # {
    #   "scenario_name": "...",
    #   "headline": "...",
    #   "macro_story": "...",
    #   "credit_risk_impact": "...",
    #   "comparison_to_baseline": "...",
    #   "key_risks": [...],
    #   "management_actions": [...],
    #   "tone": "benign",
    #   "scenario_family": "baseline"
    # }
    llm_df = llm_df.rename(
        columns={
            "scenario_name": "scenario",
            "scenario_family": "llm_family"
        }
    )

    # Keep only the narrative-related columns
    narrative_cols = [
        "scenario",
        "headline",
        "macro_story",
        "credit_risk_impact",
        "comparison_to_baseline",
        "key_risks",
        "management_actions",
        "tone",
        "llm_family",
    ]
    # Some fields may not exist depending on how you generated JSON
    narrative_cols = [c for c in narrative_cols if c in llm_df.columns]

    merged_df = merged_df.merge(
        llm_df[narrative_cols],
        on="scenario",
        how="left",
        validate="one_to_one"
    )

    print("LLM narratives merged. Columns now:")
    print(merged_df.columns.tolist())
else:
    print(f"LLM narrative file not found at: {llm_path}")


LLM narratives merged. Columns now:
['scenario', 'family', 'mean_pd', 'EL', 'EL_change_vs_baseline_pct', 'p50_pd', 'p90_pd', 'p99_pd', 'uplift_vs_baseline_pct', 'macro_GDPC1', 'macro_UNRATE', 'macro_CPIAUCSL', 'macro_FEDFUNDS', 'macro_UNRATE_delta_qoq', 'macro_FEDFUNDS_delta_qoq', 'macro_GDPC1_delta_qoq', 'macro_inflation_qoq', 'macro_real_rate_qoq', 'macro_CPIAUCSL_delta_qoq', 'headline', 'macro_story', 'credit_risk_impact', 'comparison_to_baseline', 'key_risks', 'management_actions', 'tone', 'llm_family']


In [12]:
# === 2) Helper: pretty printer for a single scenario =============

def show_scenario(name: str):
    """Render one scenario (macro + risk + LLM narrative) as Markdown."""
    r = merged_df.loc[merged_df["scenario"] == name]
    if r.empty:
        print(f"Scenario '{name}' not found in merged_df.")
        return
    r = r.iloc[0]

    md = f"## Scenario: **{r['scenario']}**  \n"
    md += f"Family: **{r['family']}**  \n\n"

    # --- Macro block ------------------------------------------------
    macro_cols = [c for c in merged_df.columns if c.startswith("macro_")]
    if macro_cols:
        md += "### Macro Environment\n"
        for col in macro_cols:
            val = r.get(col, None)
            if pd.notna(val):
                raw_name = col.replace("macro_", "")
                if isinstance(val, (int, float, np.number)):
                    md += f"- **{raw_name}**: {float(val):,.4f}\n"
                else:
                    md += f"- **{raw_name}**: {val}\n"
        md += "\n"

    # --- Risk metrics block -----------------------------------------
    md += "### Risk Metrics\n"
    risk_cols = [
        "mean_pd",
        "p50_pd", "p90_pd", "p99_pd",
        "EL", "EL_change_vs_baseline_pct",
        "uplift_vs_baseline_pct",
    ]
    for col in risk_cols:
        if col in merged_df.columns and pd.notna(r.get(col, None)):
            val = r[col]
            if isinstance(val, (int, float, np.number)):
                md += f"- **{col}**: {float(val):,.4f}\n"
            else:
                md += f"- **{col}**: {val}\n"
    md += "\n"

    # --- LLM narrative block ----------------------------------------
    if "headline" in merged_df.columns and pd.notna(r.get("headline", None)):
        md += "### LLM Narrative\n"
        md += f"**Headline:** {r['headline']}\n\n"

        if pd.notna(r.get("macro_story", None)):
            md += f"**Macro story.** {r['macro_story']}\n\n"

        if pd.notna(r.get("credit_risk_impact", None)):
            md += f"**Credit risk impact.** {r['credit_risk_impact']}\n\n"

        if pd.notna(r.get("comparison_to_baseline", None)):
            md += f"**Comparison to baseline.** {r['comparison_to_baseline']}\n\n"

        # key_risks and management_actions might be list *or* string
        kr = r.get("key_risks", None)
        if isinstance(kr, list):
            md += "**Key risks:**\n"
            for k in kr:
                md += f"- {k}\n"
            md += "\n"
        elif isinstance(kr, str) and kr.strip():
            md += f"**Key risks:** {kr}\n\n"

        ma = r.get("management_actions", None)
        if isinstance(ma, list):
            md += "**Suggested management actions:**\n"
            for a in ma:
                md += f"- {a}\n"
            md += "\n"
        elif isinstance(ma, str) and ma.strip():
            md += f"**Suggested management actions:** {ma}\n\n"

        if pd.notna(r.get("tone", None)):
            md += f"_Narrative tone: **{r['tone']}**._\n"
    else:
        md += "### LLM Narrative\n"
        md += "*(No narrative attached — run `llm_narratives.py` or check the JSON fields.)*\n"

    display(Markdown(md))


In [13]:
# === 5) Show examples ===
show_scenario("baseline_actual")


## Scenario: **baseline_actual**  
Family: **baseline**  

### Macro Environment
- **story**: The macroeconomic environment in this scenario is characterized by moderate economic growth, a stable labor market with steady employment levels, controlled inflation close to target, and interest rates that are stable and accommodative. These conditions support consistent consumer and business activity without major disruptions. Overall, the economy is neither expanding rapidly nor experiencing significant contractions, providing a foundation of stability.

### Risk Metrics
- **mean_pd**: 0.2052
- **p50_pd**: 0.1756
- **p90_pd**: 0.3918
- **p99_pd**: 0.6181
- **EL**: 730,863,518.4847
- **EL_change_vs_baseline_pct**: 0.0000
- **uplift_vs_baseline_pct**: 0.0000

### LLM Narrative
**Headline:** The baseline_actual scenario reflects a stable macroeconomic environment with credit risk metrics aligned to expected norms.

**Macro story.** The macroeconomic environment in this scenario is characterized by moderate economic growth, a stable labor market with steady employment levels, controlled inflation close to target, and interest rates that are stable and accommodative. These conditions support consistent consumer and business activity without major disruptions. Overall, the economy is neither expanding rapidly nor experiencing significant contractions, providing a foundation of stability.

**Credit risk impact.** Under these stable macro conditions, credit risk indicators such as the mean and median probability of default remain moderate and within expected ranges. The expected loss calculated reflects typical credit performance without material stress. The tail risk metrics indicate manageable levels of potential downside. This environment supports predictable credit performance with limited volatility in losses.

**Comparison to baseline.** Since no explicit baseline scenario is available for direct comparison, the baseline_actual scenario effectively serves as the reference point with zero changes in probability of default and expected losses. This confirms its role as a standard or neutral scenario representing typical operating conditions without elevated risk. Any risk management plans can consider this the foundation upon which adverse or severe scenarios would be benchmarked.

**Key risks:**
- Moderate economic growth sustaining credit quality
- Stable labor market maintaining borrower repayment capacity
- Controlled inflation and interest rates limiting financial stress

**Suggested management actions:**
- Maintain capital adequacy aligned with baseline credit risk levels
- Continue regular monitoring of macroeconomic indicators for any shifts
- Review pricing and limit frameworks to ensure they reflect stable risk conditions

_Narrative tone: **benign**._


In [14]:
show_scenario("mild_adverse")

## Scenario: **mild_adverse**  
Family: **data_driven**  

### Macro Environment
- **GDPC1**: 18,782.2430
- **UNRATE**: 5.4333
- **CPIAUCSL**: 247.2383
- **FEDFUNDS**: 1.2033
- **UNRATE_delta_qoq**: -0.0131
- **FEDFUNDS_delta_qoq**: 0.2022
- **GDPC1_delta_qoq**: 0.0040
- **inflation_qoq**: 0.0063
- **real_rate_qoq**: 0.1542
- **story**: The macroeconomic environment under the mild adverse scenario shows real GDP growth at a steady pace of 0.4% quarterly, indicating moderate expansion. The labor market is improving with a slight decline in the unemployment rate to 5.43%. Inflation remains contained with a quarterly rate of 0.63%, while the CPI level increases moderately. The policy rate rises by 20 basis points to 1.20%, reflecting gradual monetary tightening, and real interest rates remain positive but low.

### Risk Metrics
- **mean_pd**: 0.2643
- **p50_pd**: 0.2542
- **p90_pd**: 0.4187
- **p99_pd**: 0.5674
- **EL**: 941,351,089.2961
- **EL_change_vs_baseline_pct**: 28.7998
- **uplift_vs_baseline_pct**: 28.7998

### LLM Narrative
**Headline:** A mild adverse scenario characterized by modest GDP growth, improving labor market, moderate inflation, and rising interest rates leads to increased credit risk and expected losses.

**Macro story.** The macroeconomic environment under the mild adverse scenario shows real GDP growth at a steady pace of 0.4% quarterly, indicating moderate expansion. The labor market is improving with a slight decline in the unemployment rate to 5.43%. Inflation remains contained with a quarterly rate of 0.63%, while the CPI level increases moderately. The policy rate rises by 20 basis points to 1.20%, reflecting gradual monetary tightening, and real interest rates remain positive but low.

**Credit risk impact.** Despite the moderate economic growth and improving labor market, credit risk indicators show a noticeable increase. The mean probability of default rises by nearly 29% relative to baseline, driven primarily by macroeconomic uncertainties and interest rate increases. Expected losses correspondingly increase by approximately 29%, signaling higher potential credit losses for the bank. The tail and extreme tail PDs indicate elevated risk for more vulnerable segments of the portfolio, though the rise is less severe compared to extreme stress scenarios.

**Comparison to baseline.** Compared to the baseline scenario, this mild adverse scenario reflects a clear deterioration in credit risk with PD and expected losses increasing by nearly 29%. While the baseline assumes a more stable economic environment, the current scenario’s combination of modest rate hikes and inflation pressures results in elevated risk. However, the severity remains moderate given positive GDP growth and declining unemployment. Credit metrics worsen but remain within manageable levels relative to more severe scenarios.

**Key risks:**
- Rising policy interest rates contributing to borrower stress
- Moderate inflation maintaining cost pressures
- Slower improvement in labor market despite declining unemployment
- Uncertainty in economic growth despite positive GDP expansion

**Suggested management actions:**
- Review credit underwriting criteria to adjust for increased default risk
- Enhance portfolio monitoring especially segments sensitive to interest rate rises
- Consider maintaining capital buffers to absorb higher expected losses

_Narrative tone: **cautious**._


In [15]:
show_scenario("Fed_Severe")

## Scenario: **Fed_Severe**  
Family: **Fed**  

### Macro Environment
- **GDPC1**: 16,907.1878
- **UNRATE**: 6.9000
- **CPIAUCSL**: 254.7416
- **FEDFUNDS**: 0.1000
- **UNRATE_delta_qoq**: 1.5000
- **FEDFUNDS_delta_qoq**: 0.0000
- **GDPC1_delta_qoq**: -0.0890
- **inflation_qoq**: 0.0090
- **real_rate_qoq**: -0.0090
- **story**: This scenario depicts a contracting economy with Real GDP declining by approximately 8.9% quarterly, indicating a deep recession. The labor market deteriorates significantly, as the unemployment rate rises to 6.9%, increasing by 1.5 percentage points in the quarter. Inflation remains low but positive at 0.9% quarterly, while policy interest rates stay near zero with no easing or tightening. Real interest rates are slightly negative, reflecting accommodative monetary policy amid weakening economic activity.

### Risk Metrics
- **mean_pd**: 0.2467
- **p50_pd**: 0.2359
- **p90_pd**: 0.3948
- **p99_pd**: 0.5429
- **EL**: 878,670,415.1603
- **EL_change_vs_baseline_pct**: 20.2236
- **uplift_vs_baseline_pct**: 20.2236

### LLM Narrative
**Headline:** The Fed_Severe scenario projects a sharp economic downturn with rising unemployment and subdued inflation, resulting in notably higher credit risk and expected losses compared to baseline.

**Macro story.** This scenario depicts a contracting economy with Real GDP declining by approximately 8.9% quarterly, indicating a deep recession. The labor market deteriorates significantly, as the unemployment rate rises to 6.9%, increasing by 1.5 percentage points in the quarter. Inflation remains low but positive at 0.9% quarterly, while policy interest rates stay near zero with no easing or tightening. Real interest rates are slightly negative, reflecting accommodative monetary policy amid weakening economic activity.

**Credit risk impact.** The economic stress translates into elevated credit risk, with the mean probability of default increasing by over 20% relative to baseline. Expected credit losses rise commensurately, reflecting more widespread borrower distress. The median and tail probability of default metrics confirm that credit deterioration spans from typical to stressed portfolios. Notably, the extreme tail PD declines somewhat, indicating a slightly less severe impact on the most resilient segments. Overall, heightened unemployment and GDP contraction drive the increase in credit risk.

**Comparison to baseline.** Compared to the baseline scenario, which assumes a more stable economic environment, Fed_Severe exhibits a significant increase in average and median probability of default as well as expected losses. While tail PD metrics show small changes, the overall credit risk profile is meaningfully more adverse. The policy rate remains unchanged and low, indicating the scenario’s stress arises predominantly from real economy factors rather than monetary policy shocks.

**Key risks:**
- Rapid GDP contraction of nearly 9% quarterly
- Substantial rise in unemployment by 1.5 percentage points
- Sustained low inflation, limiting policy maneuverability
- Elevated probability of default and expected credit losses

**Suggested management actions:**
- Strengthen capital buffers to absorb higher expected losses under stress
- Enhance credit monitoring especially for sectors vulnerable to economic contraction
- Review and tighten credit limits and underwriting standards in high-risk portfolios

_Narrative tone: **severe**._


In [16]:
show_scenario("GenAI_S7")

## Scenario: **GenAI_S7**  
Family: **GenAI**  

### Macro Environment
- **GDPC1**: 18,112.6172
- **UNRATE**: 6.8180
- **CPIAUCSL**: 232.2177
- **FEDFUNDS**: 0.4230
- **UNRATE_delta_qoq**: -0.0061
- **FEDFUNDS_delta_qoq**: 0.0755
- **GDPC1_delta_qoq**: 0.0067
- **inflation_qoq**: 0.0039
- **real_rate_qoq**: 0.0716
- **CPIAUCSL_delta_qoq**: 0.0039
- **story**: The macroeconomic environment features modest real GDP growth of 0.67% quarterly, indicating steady but slow expansion. The labor market shows a slight improvement with unemployment decreasing to 6.82%, though it remains elevated. Inflation is moderate, with a quarterly CPI increase of 0.39%, aligning with a similar quarterly inflation rate. Interest rates have increased modestly, with the policy rate rising to 0.423% and a real interest rate around 7.16 basis points, reflecting gradual monetary tightening.

### Risk Metrics
- **mean_pd**: 0.3630
- **p50_pd**: 0.3595
- **p90_pd**: 0.5426
- **p99_pd**: 0.6836
- **EL**: 1,292,575,548.7387
- **EL_change_vs_baseline_pct**: 76.8559
- **uplift_vs_baseline_pct**: 76.8559

### LLM Narrative
**Headline:** GenAI_S7 scenario presents a mild growth environment with moderate unemployment and inflation, resulting in significantly elevated credit risk compared to baseline.

**Macro story.** The macroeconomic environment features modest real GDP growth of 0.67% quarterly, indicating steady but slow expansion. The labor market shows a slight improvement with unemployment decreasing to 6.82%, though it remains elevated. Inflation is moderate, with a quarterly CPI increase of 0.39%, aligning with a similar quarterly inflation rate. Interest rates have increased modestly, with the policy rate rising to 0.423% and a real interest rate around 7.16 basis points, reflecting gradual monetary tightening.

**Credit risk impact.** Despite the mild growth and improving labor conditions, the scenario results in a notably higher credit risk profile. The mean probability of default rises sharply by nearly 77% compared to baseline, indicating heightened borrower stress. Expected loss increases proportionally, reflecting both greater default likelihood and loss severity. These elevated risks suggest that credit portfolios may be vulnerable to underlying economic pressures even in a non-severe macro environment.

**Comparison to baseline.** Compared to the baseline scenario, GenAI_S7 exhibits a substantially higher default risk and expected loss, with mean PD increasing from 20.5% to 36.3%. While the macro environment is not heavily deteriorated, the jump in credit risk metrics indicates stress in specific borrower segments or exposures. This divergence underscores the scenario’s role as a moderate stress test rather than a benign forecast.

**Key risks:**
- Elevated unemployment rate despite slight quarterly improvement
- Moderate inflation pressures sustaining borrower cost challenges
- Gradual increase in policy and real interest rates impacting debt servicing

**Suggested management actions:**
- Review and tighten credit underwriting standards for vulnerable segments
- Increase capital buffers to absorb higher expected losses
- Enhance monitoring of loan performance and early warning indicators

_Narrative tone: **adverse**._


In [17]:
show_scenario("GenAI_S8")

## Scenario: **GenAI_S8**  
Family: **GenAI**  

### Macro Environment
- **GDPC1**: 18,123.1445
- **UNRATE**: 6.6947
- **CPIAUCSL**: 232.6304
- **FEDFUNDS**: 0.4359
- **UNRATE_delta_qoq**: -0.0182
- **FEDFUNDS_delta_qoq**: -0.0555
- **GDPC1_delta_qoq**: 0.0017
- **inflation_qoq**: 0.0060
- **real_rate_qoq**: -0.0615
- **CPIAUCSL_delta_qoq**: 0.0060
- **story**: The scenario reflects a marginally improving economic outlook with real GDP showing a slight quarterly increase and the unemployment rate declining. Inflation remains low but positive, supported by a modest quarterly CPI increase. Interest rates have decreased, with the federal funds policy rate falling, resulting in a slightly negative real interest rate. Overall, the macro environment suggests a mild recovery phase with contained inflation and accommodative monetary policy.

### Risk Metrics
- **mean_pd**: 0.2419
- **p50_pd**: 0.2309
- **p90_pd**: 0.3881
- **p99_pd**: 0.5360
- **EL**: 861,610,616.2051
- **EL_change_vs_baseline_pct**: 17.8894
- **uplift_vs_baseline_pct**: 17.8894

### LLM Narrative
**Headline:** Moderate macroeconomic improvement drives a modest rise in credit risk relative to baseline.

**Macro story.** The scenario reflects a marginally improving economic outlook with real GDP showing a slight quarterly increase and the unemployment rate declining. Inflation remains low but positive, supported by a modest quarterly CPI increase. Interest rates have decreased, with the federal funds policy rate falling, resulting in a slightly negative real interest rate. Overall, the macro environment suggests a mild recovery phase with contained inflation and accommodative monetary policy.

**Credit risk impact.** Despite the improving macro backdrop, credit risk indicators show a noticeable rise in default probabilities and expected losses. The mean probability of default increased by approximately 17.9%, indicating heightened borrower vulnerabilities. Expected losses have also risen by a similar margin, reflecting increased credit stress possibly due to lingering weaknesses in certain sectors. The tail risk metrics, however, exhibit a slight reduction or remain stable, suggesting limited extreme outcomes under this scenario.

**Comparison to baseline.** Compared to the baseline, this scenario presents moderately increased average credit risk with higher mean and median PDs alongside elevated expected loss. However, extreme tail risks are somewhat lower than baseline levels, indicating the scenario is more focused on broad credit deterioration rather than severe stress events. Overall, risk is elevated but not dramatically divergent from the baseline.

**Key risks:**
- Moderate decline in unemployment driving credit risk adjustments.
- Marginal GDP growth insufficient to fully offset credit vulnerabilities.
- Lower policy interest rates reducing borrowing costs but not eliminating credit risk buildup.

**Suggested management actions:**
- Enhance credit portfolio monitoring to identify emerging vulnerabilities in segments sensitive to moderate economic shifts.
- Review capital adequacy to ensure resilience against elevated expected losses and increased mean default probabilities.
- Adjust loan pricing and risk limits to reflect the modestly increased credit risk environment.

_Narrative tone: **cautious**._
